# Enhancement

## Sentence Retrival

#### Preprocessing 

In [1]:
import math,numpy,json,re,nltk,csv,time,re,os.path,sys,ast,itertools,string
from operator import add
from math import sqrt
from numpy import multiply
from nltk import FreqDist, DictionaryProbDist
from nltk.tokenize import word_tokenize,RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer as WNL
from nltk.tag import StanfordNERTagger
from nltk.parse.stanford import StanfordDependencyParser
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn import metrics
from collections import defaultdict,OrderedDict
from nltk import RegexpParser


stopwords = set(nltk.corpus.stopwords.words('english')) # wrap in a set() (see below)

#### Load data from json file

In [2]:
t0 = time.time()

test_path = os.path.abspath('data/QA_test.json')
dev_path = os.path.abspath('data/QA_dev.json')
train_path = os.path.abspath('data/QA_train.json')

def load_jsonfile(filepath):
    dataset = []
    with open(filepath) as jsonfile:
        for line in jsonfile:
            dataset += (json.loads(line))
    return dataset


dataset_dev = load_jsonfile(dev_path)
dataset_test = load_jsonfile(test_path)
dataset_train = load_jsonfile(train_path)
print "Import Successful "
print "There are totally", len(dataset_dev),'documents in this dev dataset'
print "There are totally", len(dataset_test),'documents in this test dataset'
print "There are totally", len(dataset_train),'documents in this  train dataset'

Import Successful 
There are totally 40 documents in this dev dataset
There are totally 42 documents in this test dataset
There are totally 360 documents in this  train dataset


#### Semantic Enhancement - Re-Build BM25

In [136]:
class BM25_p_Model(object): 
    def __init__(self, document_collection, k1=1.5, b=0.75, k3=0.0, EPS=0.25, tokenizer=my_tokenizer): 
        self.tokenizer = tokenizer 
        self.document_collection_length = len(document_collection) 
        self.avg_doc_length = sum(map(lambda x: float(len(x)), document_collection)) / self.document_collection_length 
        self.document_collection = [self.tokenizer(doc) for doc in document_collection] 
        self.document_corpus = list(itertools.chain.from_iterable(self.document_collection))
        self.corpus_freq = FreqDist(self.document_corpus)
        self.tf = [] 
        self.df = defaultdict(int) 
        self.bm25_idf = defaultdict(float) 
        self.average_idf = -1 
        self.k1 = k1 
        self.k3 = k3 
        self.EPSILON = EPS 
        self.b = b 
        self.inverted_index = defaultdict(list) 
        self.initialize() 
    def initialize(self):
        for index, document in enumerate(self.document_collection): 
            doc_term_freq = FreqDist(document) 
            self.tf.append(doc_term_freq) 
            for word, freq in doc_term_freq.items(): 
                self.df[word] += 1 
                self.inverted_index[word].append(index) 
        for word, freq in self.df.items(): 
            self.bm25_idf[word] = math.log(self.document_collection_length - freq + 0.5) - math.log(freq + 0.5) 
            
            self.average_idf = sum(map(lambda k: float(self.bm25_idf[k]), self.bm25_idf.keys())) / len(self.bm25_idf.keys()) 
    
    def predict(self, queryX, limit=1): 
        q_prediction = [] 
        for index,query in enumerate(queryX): 
            answers = self.bm25_get_most_relevant(query)[:limit] 
            if len(answers) == 0:
                previous_answer = self.bm25_get_most_relevant(queryX[index-1])[:limit] 
                q_prediction.append([i[0] for i in previous_answer]) 
            else:
                q_prediction.append([i[0] for i in answers]) 
        return q_prediction 

    def bm25_get_most_relevant(self, query): 
        query_tks = self.tokenizer(query) 
        scores = defaultdict(float)
        new_query_tks = []
        for i in query_tks:
            if i not in self.inverted_index.keys():
                new_query_tks.extend(lemma_words(i))
            else:
                new_query_tks.append(i)
        new_query_tks = list(set(new_query_tks))
        for q_token in new_query_tks: 
            for doc_index in self.inverted_index[q_token]: 
                idf = self.bm25_idf[q_token] if self.bm25_idf[q_token] >= 0 else self.EPSILON * self.average_idf 
                top = self.tf[doc_index][q_token] * (self.k1 + 1) 
                below = self.tf[doc_index][q_token] + self.k1 * (1 - self.b + self.b * self.document_collection_length / self.avg_doc_length) 
                frq_q_t = self.corpus_freq[q_token]
                scores[doc_index] += idf * top / below *(self.k3 +1)*frq_q_t/(self.k3+frq_q_t)
        prels = scores.items() 
        sorted_socres = sorted(prels, key=lambda (k, v): v, reverse=True) 
        return sorted_socres 

In [139]:
def write_csv(csv_name,model_name,dataset,limit=1):
    csv_file = open(csv_name, mode='w',)
    fieldnames = ['document_ID', 'question_ID','question','prediction_ID','prediction_sentence']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames, )
    writer.writeheader()

    ddi = 0
    for document in dataset:
        evaluate_row = []
        questions = [i['question'] for i in document['qa']]
        model = document[model_name]
        predictions = model.predict(questions,limit)
        quest_index = 0
        for pred_index in predictions:
            drow = dict()
            drow['question_ID'] = quest_index
            drow['prediction_ID'] = pred_index
            evaluate_row.append(drow)
            quest_index += 1
        doc_sents = document['sentences']
        for r in evaluate_row:
            r['document_ID'] = ddi
            r['question'] = questions[r['question_ID']].encode('utf-8')
            if len(r['prediction_ID']) != 0:
                r['prediction_sentence'] = doc_sents[r['prediction_ID'][0]].encode('utf-8')
            else:
                print 'error prediction',ddi,r['question_ID'],r['question']
            writer.writerow(r)
        ddi += 1

In [140]:
#Build models for test data set
for document in dataset_test:
    document_collections = document['sentences']
    document['bm25_model'] = BM25_p_Model(document_collections)
# write to a CSV file for test data predictions
write_csv('data/bm25_test_P_predictions.csv','bm25_model',dataset_test,1)

#### Word2Vec Query Classification

In [7]:
def load_csv(filename):
    with open(filename) as csvfile:
        readCSV = csv.DictReader(csvfile, delimiter=',')
        query = []
        query_type = []
        for row in readCSV:
            query.append(row['question'].decode('utf-8'))
            query_type.append(row['question_type'].decode('utf-8'))
    return query,query_type

train_queryX,train_type = load_csv('train_question_tag.csv')
dev_queryX,dev_type=load_csv('dev_question_tag.csv')

In [8]:
# Import various modules for string cleaning
def query_tokenizer(doc, remove_stopwords=False ):
    terms = set()
    tokenizer = RegexpTokenizer(r'\w+(?:[-./]\w+)?')
    for token in tokenizer.tokenize(doc):
        if remove_stopwords:
            if token not in stopwords: # 'in' and 'not in' operations are much faster over sets that lists
                terms.add(stemmer.stem(token.lower()))
        else:
            terms.add(stemmer.stem(token.lower()))
    return list(terms)

In [48]:
print "Parsing sentences from training set"
sentences = []
for query in train_queryX:
    sentences.append(query_tokenizer(query, remove_stopwords=True))
sentences_test = []
print "Parsing sentences from test set"
for review in dev_queryX:
    sentences_test.append(query_tokenizer(query, remove_stopwords=True))
    
print'Train sentences:' ,len(sentences)
print'Test sentences:' ,len(sentences_test)

Parsing sentences from training set
Parsing sentences from test set
Train sentences: 70159
Test sentences: 8463


In [141]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 150    # Word vector dimensionality                      
min_word_count = 20   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
import gensim
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)
#model =word2vec.Word2Vec(sentences,size=100, min_count=5, workers=2, iter=50)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)
print "Finished!"

Training model...
Finished!


In [142]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float64")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float64")
    # 
    # Loop through the reviews
    for review in reviews:
        # Print a status message every 1000th review
        if counter%1000 == 0:
            print "Review %d of %d" % (counter, len(reviews))
        # 
        # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
            
        # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

In [143]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for query in train_queryX:
     clean_train_reviews.append(query_tokenizer(query,True))

trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features )

print "Creating average feature vecs for test reviews"
clean_test_reviews = []
for query in dev_queryX:
    clean_test_reviews.append(query_tokenizer(query,True))

testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features )

Review 0 of 70159
Review 1000 of 70159


/Users/qilong/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: invalid value encountered in divide


Review 2000 of 70159
Review 3000 of 70159
Review 4000 of 70159
Review 5000 of 70159
Review 6000 of 70159
Review 7000 of 70159
Review 8000 of 70159
Review 9000 of 70159
Review 10000 of 70159
Review 11000 of 70159
Review 12000 of 70159
Review 13000 of 70159
Review 14000 of 70159
Review 15000 of 70159
Review 16000 of 70159
Review 17000 of 70159
Review 18000 of 70159
Review 19000 of 70159
Review 20000 of 70159
Review 21000 of 70159
Review 22000 of 70159
Review 23000 of 70159
Review 24000 of 70159
Review 25000 of 70159
Review 26000 of 70159
Review 27000 of 70159
Review 28000 of 70159
Review 29000 of 70159
Review 30000 of 70159
Review 31000 of 70159
Review 32000 of 70159
Review 33000 of 70159
Review 34000 of 70159
Review 35000 of 70159
Review 36000 of 70159
Review 37000 of 70159
Review 38000 of 70159
Review 39000 of 70159
Review 40000 of 70159
Review 41000 of 70159
Review 42000 of 70159
Review 43000 of 70159
Review 44000 of 70159
Review 45000 of 70159
Review 46000 of 70159
Review 47000 of 70

#### Replace the Nan values in matrix

In [144]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(trainDataVecs)
Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)
fix_train =imp.transform(trainDataVecs) 

In [145]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(testDataVecs)
Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)
fix_test =imp.transform(testDataVecs) 

In [146]:
from sklearn.metrics import accuracy_score
#Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 130 )

print "Fitting a random forest to labeled training data..."
forest = forest.fit(fix_train, train_type )
print 'Preditcing'
#Test & extract results 
result = forest.predict(fix_test)
print 'accuracy:',accuracy_score(result,dev_type)

Fitting a random forest to labeled training data...
Preditcing
accuracy: 0.649178778211


In [150]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(15,), random_state=1)

clf.fit(fix_train, train_type)                         
MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False,
       epsilon=1e-08, hidden_layer_sizes=(15,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)
pred_NN = clf.predict(fix_test)
print 'accuracy:',accuracy_score(pred_NN,dev_type)

accuracy: 0.64610658159


In [148]:
# Grid Search for Algorithm Tuning
from sklearn.model_selection import GridSearchCV

# prepare a range of alpha values to test
estimators = np.array([80,90,100,110,115,120,123,125,130,140,150])

grid = GridSearchCV(estimator=forest, param_grid=dict(n_estimators=estimators))
grid.fit(fix_test, dev_type)
print(grid)
gs_n_estimators = grid.best_estimator_.n_estimators
# summarize the results of the grid search
print "\nGrid Search best score is :",(grid.best_score_)
print "Grid Search Value  gs_n_estimators :",gs_n_estimators

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=130, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': array([ 80,  90, 100, 110, 115, 120, 123, 125, 130, 140, 150])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

Grid Search best score is : 0.639844026941
Grid Search Value  gs_n_estimators : 150
